In [1]:
from presidio_analyzer import AnalyzerEngine

analyzer = AnalyzerEngine()
print("✅ Presidio Analyzer initialized successfully!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 33.0 MB/s  0:00:11:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Users/siyisong/Downloads/Data Curation/redlake/venv312/bin/python -m pip install --upgrade pip


✅ Presidio Analyzer initialized successfully!


In [2]:
text = """
Hi, my name is Alice Johnson. 
You can reach me at alice.j@example.com or (415) 555-1234. 
I live in 123 Market St, San Francisco.
"""

results = analyzer.analyze(text=text, language="en")

for r in results:
    print(f"{r.entity_type:20} | {text[r.start:r.end]}")


EMAIL_ADDRESS        | alice.j@example.com
PERSON               | Alice Johnson
LOCATION             | Market St
LOCATION             | San Francisco
URL                  | example.com
PHONE_NUMBER         | (415) 555-1234


In [5]:
import praw
import hashlib
import json
from datetime import datetime, timezone

# === Presidio (PII Detection) ===
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

# 初始化 Presidio 引擎
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

# ==== Reddit API 凭证 ====
REDDIT_CLIENT_ID = 'HTizWMyFX5jMJakFTPt70w'
REDDIT_CLIENT_SECRET = '9K8xZ7GDZKIIR5F0grk94k3gQjtvtw'
REDDIT_USER_AGENT = "script:MyRedditApp:0.1 (by u/Hefty-Yam8420)"

# ==== 初始化 Reddit 客户端 ====
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# ==== 工具函数 ====
def sha256_hash(value):
    return hashlib.sha256(value.encode('utf-8')).hexdigest()

def clean_text_with_presidio(text: str) -> str:
    """用 Presidio 检测并匿名化潜在的个人信息（PII）"""
    if not text:
        return text
    try:
        results = analyzer.analyze(text=text, language="en")
        if not results:
            return text
        anonymized = anonymizer.anonymize(text=text, analyzer_results=results)
        return anonymized.text
    except Exception as e:
        print(f"⚠️ Presidio failed on text: {e}")
        return text

# ==== 主函数 ====
def fetch_posts_with_comments(subreddit_name, limit=10):
    subreddit = reddit.subreddit(subreddit_name)
    posts, comments = [], []

    for post in subreddit.top(limit=limit):
        # 🚫 跳过被删除或移除的帖子
        if not post.selftext or post.selftext.strip().lower() in {"[removed]", "[deleted]"}:
            print(f"⏩ Skipped post {post.id} (removed or deleted)")
            continue

        # 🚀 在这里调用 Presidio 清洗帖子内容
        body_clean = clean_text_with_presidio(post.selftext)
        title_clean = clean_text_with_presidio(post.title)

        post_data = {
            'post_id': post.id,
            'title': title_clean,
            'body': body_clean,
            'subreddit': post.subreddit.display_name,
            'author_hash': sha256_hash(str(post.author)) if post.author else None,
            'created_utc': datetime.fromtimestamp(post.created_utc, tz=timezone.utc).isoformat(),
            'score': post.score,
            'num_comments': post.num_comments,
            'permalink': post.permalink,
            'fetched_at': datetime.utcnow().isoformat()
        }
        posts.append(post_data)

        # === 抓取部分评论 ===
        try:
            post.comments.replace_more(limit=0)
            for comment in post.comments.list()[:5]:
                if not comment.body or comment.body.strip().lower() in {"[removed]", "[deleted]"}:
                    continue

                # 🚀 清洗评论内容
                body_clean = clean_text_with_presidio(comment.body)

                comments.append({
                    'comment_id': comment.id,
                    'post_id': post.id,
                    'body': body_clean,
                    'author_hash': sha256_hash(str(comment.author)) if comment.author else None,
                    'created_utc': datetime.fromtimestamp(comment.created_utc, tz=timezone.utc).isoformat(),
                    'score': comment.score,
                    'parent_id': comment.parent_id,
                    'fetched_at': datetime.utcnow().isoformat()
                })
        except Exception as e:
            print(f"⚠️ Failed to fetch comments for {post.id}: {e}")

    return posts, comments

# ==== 测试运行 ====
if __name__ == "__main__":
    posts, comments = fetch_posts_with_comments("technology", limit=10)
    print(f"✅ Collected {len(posts)} posts and {len(comments)} comments.")

    # 输出前两个样本看看效果
    print(json.dumps(posts[:2], indent=2))


⏩ Skipped post 1i7gufa (removed or deleted)
⏩ Skipped post kt785i (removed or deleted)
⏩ Skipped post 7j6kn4 (removed or deleted)
⏩ Skipped post u178zp (removed or deleted)
⏩ Skipped post df1g3g (removed or deleted)
⏩ Skipped post erd274 (removed or deleted)
⏩ Skipped post t0y163 (removed or deleted)
⏩ Skipped post t4ynmb (removed or deleted)


/var/folders/pf/vw43rk0x6rz_5021jrdsnk_m0000gn/T/ipykernel_50679/3498422226.py:69: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'fetched_at': datetime.utcnow().isoformat()


⏩ Skipped post 1nimpmu (removed or deleted)
✅ Collected 1 posts and 5 comments.
[
  {
    "post_id": "7dwqv6",
    "title": "If <LOCATION> was half as verbal about net neutrality as they are about Star Wars Battlefront II, then we could stop ISP's and the FCC",
    "body": "All it takes is one call. It's our internet. \n\n**<URL>\n\n**<URL>\n\n**<URL>\n\n**<URL>\n\n**<URL>\n\n**<URL>\n\n**<URL>\n\nEDIT: thank you for my first gold(s) kind strangers. All I want is for people to be aware and take action, not spend money on me.",
    "subreddit": "technology",
    "author_hash": "25e4d480e1ddf160863af0ec63ea3645cf2d40c5a5673fb7f7bb00d83f699b0c",
    "created_utc": "2017-11-18T23:44:27+00:00",
    "score": 121056,
    "num_comments": 2508,
    "permalink": "/r/technology/comments/7dwqv6/if_reddit_was_half_as_verbal_about_net_neutrality/",
    "fetched_at": "2025-10-25T01:53:26.155329"
  }
]


/var/folders/pf/vw43rk0x6rz_5021jrdsnk_m0000gn/T/ipykernel_50679/3498422226.py:91: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'fetched_at': datetime.utcnow().isoformat()


In [6]:
test_samples = [
    # 🧍‍♀️ 人名 + 邮箱
    "Hi everyone, I'm Alice Johnson. You can contact me at alice.johnson@gmail.com for beta testing.",
    
    # 📱 电话号 + 地址
    "Please call me at (415) 555-9876, I’m living at 1200 Market Street, San Francisco.",
    
    # 💳 信用卡号
    "I accidentally posted my card number 4111 1111 1111 1111 in the last thread 😭.",
    
    # 🧾 社会安全号 (SSN)
    "My SSN is 123-45-6789, but I’ve already reported it as compromised.",
    
    # 🏢 公司邮件 + 人名混合
    "This is David Lee from Tesla, you can reach me via dlee@tesla.com if you have ideas about solar panels.",
    
    # 🇨🇳 中英文混合（检测英文实体）
    "我叫 Jack Ma，我的邮箱是 jack.ma@alibaba.com，谢谢。",
    
    # 💬 Reddit常见语句
    "Hey guys, this is John from Chicago, hit me up at +1 650 123 4567 if you want to collaborate on the app.",
    
    # 🧠 混合地址、名字与日期
    "My roommate Sarah left on 2024-08-12 from 42 Wallaby Way, Sydney.",
    
    # 📧 非典型邮箱形式（检测鲁棒性）
    "Ping me at sam(dot)roberts(at)yahoo(dot)com.",
    
    # 🔐 网络账号信息
    "My GitHub username is jason1989, and my Slack handle is @jason_dev."
]


In [7]:
def test_presidio_cleaning(samples):
    for i, text in enumerate(samples, 1):
        cleaned = clean_text_with_presidio(text)
        print(f"\n🧩 Example {i}:")
        print("Before:", text)
        print("After :", cleaned)

test_presidio_cleaning(test_samples)



🧩 Example 1:
Before: Hi everyone, I'm Alice Johnson. You can contact me at alice.johnson@gmail.com for beta testing.
After : Hi everyone, I'm <PERSON>. You can contact me at <EMAIL_ADDRESS> for beta testing.

🧩 Example 2:
Before: Please call me at (415) 555-9876, I’m living at 1200 Market Street, San Francisco.
After : Please call me at <PHONE_NUMBER>, I’m living at <LOCATION>, <LOCATION>.

🧩 Example 3:
Before: I accidentally posted my card number 4111 1111 1111 1111 in the last thread 😭.
After : I accidentally posted my card number <CREDIT_CARD> in the last thread 😭.

🧩 Example 4:
Before: My SSN is 123-45-6789, but I’ve already reported it as compromised.
After : My SSN is 123-45-6789, but I’ve already reported it as compromised.

🧩 Example 5:
Before: This is David Lee from Tesla, you can reach me via dlee@tesla.com if you have ideas about solar panels.
After : This is <PERSON> from Tesla, you can reach me via <EMAIL_ADDRESS> if you have ideas about solar panels.

🧩 Example 6:
Before